# Analyzing NYC Taxi Data

## Introduction

## Learning Objectives

## About the Dataset

## Installation

## Library Import

## Installing and Loading Extensions

## Loading Taxi Data

### Inspecting the Data

## Temporal Analysis

### Trips by Hour of Day

### As-Of Joins (Time-Aware Joins)

### Trips by Day of Week

### Peak vs Off-Peak Analysis

## Spatial Analysis

### Pickup Location Hotspots

### Trip Distance Distribution

### Geographic Distribution Using H3

## Trip Flow Analysis

### Top Origin-Destination Pairs

### Airport Trips Analysis

## Payment and Economic Analysis

### Payment Type Analysis

### Tipping Patterns

### Fare per Mile Analysis

## Passenger Behavior Analysis

## Multi-Month Analysis

## Visualization

## Performance Optimization Tips

### Filter Early

### Use Parquet Partitioning

### Sample for Exploration

## Key Takeaways

## Exercises